In [2]:
import os
import numpy as np
from datetime import datetime
from tensorflow.keras.layers import Dense, Reshape, Flatten, Conv2DTranspose
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from astropy.io import fits
import cv2

def preprocess_fit(image_path):
    with fits.open(image_path) as hdul:
        image_data = hdul[0].data.astype(np.uint8)
        resized_image = cv2.resize(image_data, (224, 224))
        rgb_image = cv2.cvtColor(resized_image, cv2.COLOR_GRAY2RGB)
        preprocessed_image = np.expand_dims(rgb_image, axis=0)
    return preprocessed_image

def generate_augmented_data(generator, num_samples):
    # Generate random noise as input to the generator
    random_input = np.random.randn(num_samples, 100)
    # Generate synthetic FIT files using the generator
    generated_images = generator.predict(random_input)
    return generated_images

folder_path = './Data/fits/'

# Define GAN architecture
generator = Sequential([
    Dense(7*7*256, input_shape=(100,), activation='relu'),
    Reshape((7, 7, 256)),
    Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', activation='relu'),
    Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', activation='relu'),
    Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', activation='sigmoid')
])

# Sample usage: Load a FIT file and preprocess it
sample_fit_file = os.path.join(folder_path, 'tic1.fit')
preprocessed_image = preprocess_fit(sample_fit_file)

# Sample usage: Generate augmented data using the GAN
num_samples = 10  # Adjust the number of samples according to your needs
generated_data = generate_augmented_data(generator, num_samples)

# Sample usage: Save the generated data (for demonstration purposes)
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
for i, image in enumerate(generated_data):
    image_filename = f"generated_image_{i}_{current_datetime}.png"
    cv2.imwrite(image_filename, image)
    print(f"Generated image {i} saved as {image_filename}")


1/1 [==============================] - 0s 327ms/step
Generated image 0 saved as generated_image_0_2024-04-23_08-17-29.png
Generated image 1 saved as generated_image_1_2024-04-23_08-17-29.png
Generated image 2 saved as generated_image_2_2024-04-23_08-17-29.png
Generated image 3 saved as generated_image_3_2024-04-23_08-17-29.png
Generated image 4 saved as generated_image_4_2024-04-23_08-17-29.png
Generated image 5 saved as generated_image_5_2024-04-23_08-17-29.png
Generated image 6 saved as generated_image_6_2024-04-23_08-17-29.png
Generated image 7 saved as generated_image_7_2024-04-23_08-17-29.png
Generated image 8 saved as generated_image_8_2024-04-23_08-17-29.png
Generated image 9 saved as generated_image_9_2024-04-23_08-17-29.png
